<font face="Verdana, cursive, sans-serif" >
<center><H1>Quickly Generate Missing/Invalid Reports with SAS macros</H1></center>

<center><H2><font color='darkred'>How to quickly assess 500+ variables in less than 1 minute?</font></H2></center>

<p>This documentation is powered by <b>Jupyter Notebooks</b>. To learn more about how to code SAS in Jupyter Notebooks environment, please refer to <a href="https://github.com/sassoftware/sas_kernel">SAS Kernel for Jupyter</a>. Please note that you - DO NOT -  required to have Python or Jupyter Notebooks in order to utilise these <b>SAS</b> macros.


<p>This program demonstrate how to quickly assess data completeness using SAS macros. There are 2 macros in this notebook
 <ol>
<li><code>MISSING_REPORT</code>, a macro that calculate %missing and %populated of all variables in 1 single dataset. The output from this macro is a new dataset with prefix <i>MSREPORT__</i>
</li>
<li><code>MISSING_REPORT_ALL</code>, a macro that calculate %missing and %populated  of all variables in from multiple datasets. The output from this macro is a new dataset <i>MSREPORT_ALL</i>, unless specified new output name.
    </li>
</ol>

<p><b>Credits:</b>
<p><code>MISSING_REPORT</code> code is partially from Mike Zdeb, I am in deep gratitude for his sharing in <a href="https://www.sas.com/content/dam/SAS/en_ca/User%20Group%20Presentations/TASS/Zdeb-MissingData.pdf">SAS forums</a>. <code>MISSING_REPORT_ALL</code> is extended capability version of Zdeb's work, by utilising the power of <code>DO_OVER</code> macro, published by <a href="http://www2.sas.com/proceedings/sugi31/040-31.pdf">Ted Clay</a>
    

<H3><font color='darkred'>What is Data Completeness?</font></H3>
<p>Data completeness is one of data quality assessment key metrics. It is basically asking a simple question, how complete is the data? How well is it populated. One simple way to look at it is to measure 'missing/empty' vs. 'not missing'.  However, even 'not missing' does not always mean 'complete'. Take look at this mock up data.
    
<img src="./images/msreport_sample_1.png" >
    
<p>As illustrated, darkred highlighted cells are either 'missing/empty' or 'invalid'. Take <b>EMAIL</b> as an example, if we naively count 'not missing' cells, we would have concluded that <b>EMAIL</b> is 100% populated whereas it is actually 20% populated,(3/15 = 20%). From the above illustrated figure, the summary of %populated of all variables are shown as follows:-
    
<img src="./images/msreport_sample_2.png" >
 
The patterns of missing or invalid values, if can be defined, can be captured with PROC FORMAT. The macro <code>MISSING_REPORT</code> and <code>MISSING_REPORT_ALL</code> are doing just that. By defining the pattern of 'missing' and 'invalid', we can calculate percentage of populated/missing of 1 or more datasets.



<font face="Verdana, cursive, sans-serif" >
<b>First, let's execute the macros</b>

In [ ]:
OPTION NOSOURCE NONOTES;

%LET LOC_SASMACRO=C:\sasmacro;
%INCLUDE "&LOC_SASMACRO.\NUMLIST.SAS";
%INCLUDE "&LOC_SASMACRO.\ARRAY.SAS";
%INCLUDE "&LOC_SASMACRO.\DO_OVER.SAS";
%INCLUDE "&LOC_SASMACRO.\MISSING_REPORTS.SAS";

<font face="Verdana, cursive, sans-serif" >
<b>Then, define the pattern of missing/invalid values</b>

In [ ]:
/*************************
Defined formats for missing/invalids values.
**************************/
PROC FORMAT;
VALUE NM_MISS 
    .= '0' 
    99999999= '0'
    OTHER = '1'
;
VALUE $CH_MISS 
    '',' ','.','-','*'= '0' 
    'N/A','n/a','NA','N.A','-NA-','na','n.a.','n.a' = '0'
    'NULL','null','NONE','--NONE--' = '0'
    'unknown','UNKNOWN','Z_ERROR','Z_MISSING'= '0'
    '99999999','X','TESTUSER','U','C9999'= '0'
    'email@domain.com'= '0'
    OTHER = '1'
;
VALUE $NM_MISSLABEL
    '0'="MISS/INVALID"
    '1'="POPUPATED"
;
RUN;

<font face="Verdana, cursive, sans-serif" >
<b>Then, create a mock up dataset.</b>

/*****************
Mockup data
******************/
DATA SAMPLE;
    INFILE DATALINES DSD MISSOVER DELIMITER=',';
    INFORMAT USERID$5. NAME$100. GENDER$1. BIRTHDATE$8. EFFECTIVEDATE$8. EXPIRYDATE$8. PHONE$12. EMAIL$100.;
    INPUT USERID$ NAME$ GENDER$ BIRTHDATE$ EFFECTIVEDATE$ EXPIRYDATE$ PHONE$ EMAIL$;
DATALINES;
C0001,Abel,M,99999999,20140102,20140112,NA,n.a
C0002,Maggie,F,99999999,20140105,,NA,*
C0003,John,M,99999999,20140107,20140125,NA,*
C0004,Rose,M,99999999,20140107,99999999,345-466-4467,unknown
C0005,Greoge,M,19910116,20140108,20140205,,Greoge@somedomain.com
C0006,Luisa,F,20001010,20140109,20140118,,n/a
C0007,Carol,U,20011212,99999999,20140115,345-466-2367,email@domain.com
C0008,James,U,19701212,20140112,20140115,,email@domain.com
C0009,-,U,,,20140121,345-466-4467,email@domain.com
C0010,-,U,19800725,20140115,99999999,,n/a
C0011,Beth,F,20010830,20140117,20140117,,Beth@somedomain.com
C0012,Charle,,,,20140124,345-466-4888,n.a
C0013,Michael,,,20140121,20140122,,Michael@somedomain.com
C9999,TESTUSER,U,99999999,20140126,,345-645-4467,n/a
C9999,TESTUSER,U,99999999,20140128,20140129,,n/a
;
RUN;

In [31]:
PROC PRINT DATA=SAMPLE NOOBS;RUN;

USERID,NAME,GENDER,BIRTHDATE,EFFECTIVEDATE,EXPIRYDATE,PHONE,EMAIL
C0001,Abel,M,99999999,20140102,20140112,NA,n.a
C0002,Maggie,F,99999999,20140105,,NA,*
C0003,John,M,99999999,20140107,20140125,NA,*
C0004,Rose,M,99999999,20140107,99999999,345-466-4467,unknown
C0005,Greoge,M,19910116,20140108,20140205,,Greoge@somedomain.com
C0006,Luisa,F,20001010,20140109,20140118,,n/a
C0007,Carol,U,20011212,99999999,20140115,345-466-2367,email@domain.com
C0008,James,U,19701212,20140112,20140115,,email@domain.com
C0009,-,U,,,20140121,345-466-4467,email@domain.com
C0010,-,U,19800725,20140115,99999999,,n/a


<font face="Verdana, cursive, sans-serif" >
<font color='darkred'><H3>Missing Report of 1 single dataset</H3></font>
<p><b>Macro</b> :  <code>MISSING_REPORT</code>
<p>Note that we have defined formats as input parameters for the macro,<br><i><font color='blue'> The idea is that - you can have different formats designated for different datasets</font></i>

In [ ]:
%MISSING_REPORT(DSNAME=SAMPLE,
                FMT_MISSNUM=NM_MISS.,
                FMT_MISSCHAR=$CH_MISS.); 

<font face="Verdana, cursive, sans-serif" >
By default, the output from the macro is a summary of %populated and %missing, in a prefixed-dataset

In [29]:
PROC PRINT DATA=MSREPORT_SAMPLE NOOBS LABELS; RUN;

VARIABLE,N_MISSING,%_MISSING,N_POPULATED,%_POPULATED,TYPE,LENGTH
BIRTHDATE,9,60.0,6,40.0,2,8
EFFECTIVEDATE,3,20.0,12,80.0,2,8
EMAIL,12,80.0,3,20.0,2,100
EXPIRYDATE,4,26.7,11,73.3,2,8
GENDER,8,53.3,7,46.7,2,1
NAME,4,26.7,11,73.3,2,100
PHONE,10,66.7,5,33.3,2,12
USERID,2,13.3,13,86.7,2,5


<font face="Verdana, cursive, sans-serif" >
<font color='darkred'><H3>Missing Report of multiple datasets</H3></font>
<p><b>Macro</b> :  <code>MISSING_REPORT_ALL</code>

<p>Now we specify the library that holds the datasets and the list of targeting datasets,
 <br><i><font color='blue'> With the power of <code>DO_OVER</code>, the macro will loop thru all datasets and all variables</font></i>


In [ ]:
%MISSING_REPORT_ALL(TARGET_LIB=SASHELP,
                    DSNAME_LIST=CARS CLASS BASEBALL,
                    FMT_MISSNUM=NM_MISS.,
                    FMT_MISSCHAR=$CH_MISS.,
                    REPORTNAME=MSREPORT_ALL);

In [30]:
PROC PRINT DATA=MSREPORT_ALL NOOBS LABELS; RUN;

Library Name,DSNAME,VAR,N_MISSING,%_MISSING,N_POPULATED,%_POPULATED,Variable Type,Variable Length
SASHELP,BASEBALL,CRATBAT,0,0.0,322,100.0,1,8
SASHELP,BASEBALL,CRBB,0,0.0,322,100.0,1,8
SASHELP,BASEBALL,CRHITS,0,0.0,322,100.0,1,8
SASHELP,BASEBALL,CRHOME,0,0.0,322,100.0,1,8
SASHELP,BASEBALL,CRRBI,0,0.0,322,100.0,1,8
SASHELP,BASEBALL,CRRUNS,0,0.0,322,100.0,1,8
SASHELP,BASEBALL,DIV,0,0.0,322,100.0,2,16
SASHELP,BASEBALL,DIVISION,0,0.0,322,100.0,2,8
SASHELP,BASEBALL,LEAGUE,0,0.0,322,100.0,2,8
SASHELP,BASEBALL,LOGSALARY,59,18.3,263,81.7,1,8


<font face="Verdana, cursive, sans-serif" >
<font color='darkred'><H3>Summarising the level of completeness in the dataset</H3></font>

<p>With summarised %populated dataset, we can easily take a quick look at data completeness. To makethe report readible, it is recommended to classify the range of completeness into 'traffic light' bins. 

<p>In this demonstration, %populated < 50% is 'Red', 50%-75% is 'Amber', and 75% or more is 'Green' 
<br><i><font color='blue'> First, let's take a look at granular level, %populated by each variable </font></i>
<img src="./images/rag_1.png" >

In [ ]:

PROC FORMAT;
VALUE RAG
    LOW-<50 = 'LIGHTRED'
    50-<75 = 'LIGHTORANGE'
    75 - HIGH = 'LIGHTGREEN';
 
RUN;

PROC TABULATE DATA=MSREPORT_SAMPLE MISSING ;
CLASS VAR ;
VAR P_OK ;
TABLE VAR='By Variable',P_OK='%Populated'*MEAN=''*[STYLE=[BACKGROUND=RAG.]] ;
RUN;


<font face="Verdana, cursive, sans-serif" >

<br><i><font color='blue'> Then, let's classify variables in to a group of variables. %populated by each variable group</font></i>
<img src="./images/rag_2.png" >

In [ ]:

PROC FORMAT;

VALUE $VARGROUP
    'NAME','BIRTHDATE','GENDER'='CUSTOMER DEMOGRAPHIC'
    'PHONE','EMAIL'='CUSTOMER CONTACTS'
    'EXPIRYDATE','EFFECTIVEDATE','USERID' = 'POLICY DETAILS'
    OTHER='OTHERS'
;
    
RUN;


PROC TABULATE DATA=MSREPORT_SAMPLE MISSING ;
FORMAT VAR $VARGROUP.  ;
CLASS VAR ;
VAR P_OK ;
TABLE (VAR='By Variable')ALL='Overall',(P_OK='%Populated'*(MEAN='')*[STYLE=[BACKGROUND=RAG.]]);
RUN;

